In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'
CRYPTO_TARGET = 'TINYAES128C'
SS_VER = 'SS_VER_2_1'
allowable_exceptions = None
VERSION = 'HARDWARE'

In [ ]:
%%bash -s "$PLATFORM"
cd ../../hardware/victims/firmware/simpleserial-base
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=SS_VER_2_1

In [ ]:
import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2_CDC
    else:
        raise ValueError("SS_VER_2 only!")
except:
    SS_VER="SS_VER_2_1"
    target_type = cw.targets.SimpleSerial2_CDC

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")

In [ ]:
if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
elif "neorv32" in PLATFORM.lower():
    prog = cw.programmers.NEORV32Programmer
else:
    prog = None

In [ ]:
import time
time.sleep(0.05)
scope.default_setup()
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    elif "neorv32" in PLATFORM.lower():
        raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)

In [ ]:
cw.program_target(scope, prog, "../../hardware/victims/firmware/simpleserial-base/simpleserial-base-{}.hex".format(PLATFORM))

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time
#cw.target_logger.handlers[0].setLevel(cw.logging.DEBUG)


ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

key, text = ktp.next()
target.read(10000)

target.set_key(key)


N = 25 #increase nano reliability

for i in trange(N, desc='Capturing traces'):
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()

    response = target.simpleserial_read('r', 16)
    if text != response:
        raise ValueError("Echo failed! {} vs {} i={}".format(response, text, i))
        
    if ret:
        raise OSError("Scope timed out! i={}".format(i))
    
    
    textin_array.append(text)
    
    key, text = ktp.next()
    

In [ ]:
scope.dis()
target.dis()